# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame

file = "../output_data/clean_city_data.csv"
file_df = pd.read_csv(file)
file_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Kaitangata,52,NZ,1604261772,67.0,-46.28,169.85,51.01,8.99
1,1,Vanimo,99,PG,1604261455,81.0,-2.67,141.30,78.71,4.74
2,2,Qaanaaq,0,GL,1604261640,74.0,77.48,-69.36,1.08,7.56
3,3,Rikitea,77,PF,1604261494,86.0,-23.12,-134.97,74.55,22.30
4,4,Vardø,75,NO,1604261772,86.0,70.37,31.11,39.20,26.40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Find the maximum value of Humidity to set the maximum intensity for the heatmap layer
file_df['Humidity'].max()

100.0

In [4]:
file_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,Kaitangata,52,NZ,1604261772,67.0,-46.28,169.85,51.01,8.99
1,1,Vanimo,99,PG,1604261455,81.0,-2.67,141.30,78.71,4.74
2,2,Qaanaaq,0,GL,1604261640,74.0,77.48,-69.36,1.08,7.56
3,3,Rikitea,77,PF,1604261494,86.0,-23.12,-134.97,74.55,22.30
4,4,Vardø,75,NO,1604261772,86.0,70.37,31.11,39.20,26.40


In [9]:
#  Access maps with unique API key
gmaps.configure(api_key = g_key)

# Create an empty list to store the locations
locations = []

# Use the latitude and longitude as locations
file_df['Location'] = list(zip(file_df["Latitude"], file_df["Longitude"]))

# Store latitude and longitude in locations list
for coord in file_df['Location']:
    locations.append(coord)

# To use humidity as the weight    
humidity = file_df['Humidity']

# Add Heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Narrow down the cities to fit weather conditions

    # A max temperature lower than 80 degrees but higher than 70.

    # Wind speed less than 10 mph.

    # Zero cloudiness.

    # Drop any rows that don't contain all three conditions and any rows will null values.

hotel_df = file_df[(file_df['Max Temp'] < 80) & (file_df['Max Temp'] > 70) & (file_df['Wind Speed'] < 10) & 
                 (file_df['Cloudiness'] == 0)]
hotel_df["Hotel Name"] = ""
hotel_df.dropna(inplace=True)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Narrow down the cities to fit weather conditions

    # A max temperature lower than 80 degrees but higher than 70.

    # Wind speed less than 10 mph.

    # Zero cloudiness.

    # Drop any rows that don't contain all three conditions and any rows will null values.

hotel_df = file_df[(file_df['Max Temp'] < 80) & (file_df['Max Temp'] > 70) & (file_df['Wind Speed'] < 10) & 
                 (file_df['Cloudiness'] == 0)]
hotel_df["Hotel Name"] = ""
hotel_df.dropna(inplace=True)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
# set up a parameters dictionary

params = {
    "radius": 50000,
    "keyword": "hotel",
    "key": g_key
}

# Use the latitude and longitude to identify hotels
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

# change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    hotels = requests.get(base_url, params=params)
    hotels_json = hotels.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels_json["results"][0]["name"]
    except IndexError:
        print("Name not found")

C:\Users\Owner\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Name not found


In [14]:
# Drop any empty Hotel row in the DataFrame

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head()

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,City ID,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Location,Hotel Name
16,16,Mormugao,0,IN,1604261774,80.0,15.40,73.80,77.04,3.69,"(15.4, 73.8)","Taj Hotel & Convention Centre, Goa"
36,36,Yārāda,0,IN,1604261777,88.0,17.65,83.27,78.80,2.24,"(17.65, 83.27)",Dolphin Hotel Vishakapatnam
44,44,Coyhaique,0,CL,1604261417,21.0,-45.58,-72.07,78.80,8.05,"(-45.58, -72.07)",Entre Cumbres Apart Hotel
155,155,Chhor,0,PK,1604261797,18.0,25.52,69.77,77.68,7.25,"(25.52, 69.77)",Ghosia Guest House
196,196,Noumea,0,NC,1604261255,57.0,-22.28,166.46,79.00,2.24,"(-22.28, 166.46)",Nouvata


In [15]:
# NOTE: Do not change any of the code in this cell
# hotel_locations = [hotel for hotel in hotel_df['Hotel Name']]

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display figure

humidity= file_df['Humidity']
location2 = file_df['Location']
# # fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location2, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))